4/7 - this is the current code to download WorldPop population density data for year 2020 (most recent year at resolution = 1km)

This will not require frequent re-updates, but any updates will require changing several of the below code cells (see notes in comments and in Markdown cells below)

In [1]:
import requests
import json
import pandas as pd
from urllib.request import urlretrieve

In [19]:
# view available aliases (also serves as test that API endpoint is working correctly)
url = "https://www.worldpop.org/rest/data/pop/"

response = requests.get(url)

text = json.loads(response.text)
text

{'data': [{'alias': 'pic', 'name': 'Individual countries'},
  {'alias': 'pop_continent', 'name': 'Whole Continent'},
  {'alias': 'wpgp',
   'name': 'Unconstrained individual countries 2000-2020 ( 100m resolution )'},
  {'alias': 'wpgp1km',
   'name': 'Unconstrained global mosaics 2000-2020 ( 1km resolution )'},
  {'alias': '', 'name': 'WP00643'},
  {'alias': 'wpgpunadj',
   'name': 'Unconstrained individual countries 2000-2020 UN adjusted ( 100m resolution )'},
  {'alias': 'wpic1km',
   'name': 'Unconstrained individual countries 2000-2020  ( 1km resolution )'},
  {'alias': 'wpicuadj1km',
   'name': 'Unconstrained individual countries 2000-2020 UN adjusted ( 1km resolution )'},
  {'alias': 'cic2020_100m',
   'name': 'Constrained Individual countries 2020 ( 100m resolution )'},
  {'alias': 'cic2020_UNadj_100m',
   'name': 'Constrained Individual countries 2020 UN adjusted  (100m resolution)'},
  {'alias': 'G2_UC_POP_2024_100m',
   'name': 'Unconstrained individual countries 2024 ( 100m 

The above shows the available resolutions and time periods for population density data. Right now we are using `wpic1km`, but if a preferred version becomes available, we can update the below cell which specifies the alias. The code below automatically updates that alias in the API call.

In [20]:
alias = 'wpic1km'

In [15]:
# read in country metadata
metadata = pd.read_excel('../Plan-EO_Country_meta-data.xlsx')
metadata = metadata[['Name', 'ISO2', 'ISO3']] # get the sections we need
metadata.Name = metadata.Name.str.replace(' ', '_') # to match Name to the format of the Individual_country_data folder
metadata.head()

,Name,ISO2,ISO3
0,Afghanistan,AF,AFG
1,Algeria,DZ,DZA
2,Angola,AO,AGO
3,Argentina,AR,ARG
4,Armenia,AM,ARM


This loop may need to be updated, specifically the lines handling extraction of the desired URL (beginning with `tiff_urls = ...`) if the desired alias (see above) needs to be updated. I will provide a walkthrough of how exactly to determine the necessary changes below, for documentation purposes.

In [14]:
# loop through every country
for i in range(len(metadata)):
    
    # extract iso3 and country name
    iso3 = metadata.loc[i,'ISO3']
    country = metadata.loc[i,'Name']

    # set query URL (we are using worldpop's REST API endpoint)
    url = f'https://www.worldpop.org/rest/data/pop/{alias}?iso3={iso3}'

    # load response and extract data
    response = requests.get(url)
    text = json.loads(response.text)
    data = text['data']

    # extract URL path to TIFF folder (note that this may change if the alias changes - potential for different file structure)
    # but stable for now
    tiff_urls = [item for item in data if item.get('popyear') == '2020'][0]['files']
    tiff_url = [item for item in tiff_urls if item.endswith('.tif')][0]
    
    # get end path (name at which the extracted TIFF will be stored)
    end_path = tiff_url.split('/')[-1]
    # option to print end_path to track progress
    # print(end_path)
    
    # set path to destination folder and retrieve TIFF file from data index, store in destination folder
    path = f'../Individual_country_data/{iso3}_{country}/04_Population_density_data/World_pop/{end_path}'
    urlretrieve(tiff_url, path)

Burundi
Cambodia
Cameroon
Cape_Verde
Central_African_Republic
Chad
China
Colombia
Comoros
Congo
Costa_Rica
Cote_d'Ivoire
Cuba
Democratic_Republic_of_Congo
Djibouti
Dominica
Dominican_Republic
Ecuador
Egypt
El_Salvador
Equatorial_Guinea
Eritrea
Eswatini
Ethiopia
Fiji
Gabon
Gambia
Georgia
Ghana
Grenada
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
India
Indonesia
Iran
Iraq
Jamaica
Jordan
Kazakhstan
Kenya
Kiribati
Kyrgyzstan
Laos
Lebanon
Lesotho
Liberia
Libya
Madagascar
Malawi
Malaysia
Maldives
Mali
Marshall_Islands
Mauritania
Mauritius
Mexico
Micronesia
Mongolia
Morocco
Mozambique
Myanmar
Namibia
Nepal
Nicaragua
Niger
Nigeria
North_Korea
Pakistan
Palestine
Panama
Papua_New_Guinea
Paraguay
Peru
Philippines
Rwanda
Saint_Lucia
Saint_Vincent_and_the_Grenadines
Samoa
Sao_Tome_and_Principe
Senegal
Sierra_Leone
Solomon_Islands
Somalia
South_Africa
South_Sudan
Sri_Lanka
Sudan
Suriname
Syria
Tajikistan
Tanzania
Thailand
Timor-Leste
Togo
Tonga
Tunisia
Turkey
Turkmenistan
Tuvalu
Uganda
Uzbek